# Actividad 1 – Identificación de la recurrencia de un algoritmo recursivo

El objetivo de esta actividad es **aprender a derivar la ecuación de recurrencia** que modela el tiempo de ejecución de un algoritmo recursivo de tipo **divide y vencerás**.

---

## 0. Recordatorio teórico

Cuando analizamos un algoritmo recursivo, su tiempo de ejecución se expresa a menudo mediante una **ecuación de recurrencia** de la forma:

$$
T(n) = a \cdot T(n/b) + f(n),
$$

donde:

- **n** es el tamaño del problema,
- **a** es el número de subproblemas generados en cada llamada,
- **b** es el factor de reducción del tamaño del problema en cada subproblema,
- **f(n)** es el coste del trabajo adicional que se realiza fuera de las llamadas recursivas (dividir el problema, combinar resultados, operaciones constantes, etc.).

En particular, los algoritmos que dividen el problema en **dos subproblemas de tamaño n/2** y hacen un trabajo constante adicional suelen dar lugar a recurrencias del tipo:

$$
T(n) = 2T(n/2) + c.
$$

Este tipo de análisis es la base para aplicar después técnicas como el **Teorema Maestro**, pero en esta actividad nos centraremos en el **paso previo esencial**: **identificar correctamente la recurrencia a partir del pseudocódigo**.


## 1. Definición del problema y del algoritmo

Vamos a analizar el siguiente algoritmo recursivo `MergeCount`, que cuenta cuántos elementos positivos hay en un vector, dividiendo el problema en dos mitades:

```text
function MergeCount(a, left, right):
    if left == right:
        return 1 if a[left] > 0 else 0
    mid = (left + right) // 2
    return MergeCount(a, left, mid) + MergeCount(a, mid+1, right)
```

La idea es:

- Si el segmento tiene un único elemento (`left == right`), se devuelve `1` si es positivo y `0` en caso contrario.
- Si el segmento tiene más de un elemento, se divide el problema en **dos mitades**:
  - Parte izquierda: de `left` a `mid`
  - Parte derecha: de `mid+1` a `right`
- Se suman los resultados de las dos llamadas recursivas.

Definimos el tamaño del problema como:

$$
n = \text{right} - \text{left} + 1.
$$

El objetivo de esta actividad es **derivar la ecuación de recurrencia T(n)** que describe el tiempo de ejecución de este algoritmo.


## 2. Implementación en Python

Implementamos el algoritmo `MergeCount` en Python, manteniendo exactamente la misma lógica que en el pseudocódigo, y añadimos una función envoltorio para trabajar sobre el vector completo.


In [1]:
def merge_count(a, left, right):
    """Cuenta cuántos elementos positivos hay en a[left:right+1]
    usando un esquema recursivo de divide y vencerás.
    """
    if left == right:
        return 1 if a[left] > 0 else 0
    
    mid = (left + right) // 2
    return merge_count(a, left, mid) + merge_count(a, mid + 1, right)


def count_positives(a):
    """Función de alto nivel: cuenta positivos en todo el vector a."""
    if not a:
        return 0
    return merge_count(a, 0, len(a) - 1)


Probamos el algoritmo con un ejemplo sencillo para comprobar su funcionamiento:


In [2]:
ejemplo = [1, -2, 3, 0, 5, -1]
print("Vector:", ejemplo)
print("Número de positivos:", count_positives(ejemplo))

Vector: [1, -2, 3, 0, 5, -1]
Número de positivos: 3


## 3. Modelo de coste: definición de T(n)

Definimos $T(n)$ como el **tiempo de ejecución** de `merge_count(a, left, right)` cuando el segmento `a[left:right+1]` tiene tamaño $n$.

Analizamos los dos casos del algoritmo:

### 3.1. Caso base (n = 1)

Si `left == right`, el segmento tiene un único elemento. El algoritmo ejecuta:

- Una comparación (`a[left] > 0`).
- Una decisión y un retorno.

Ese trabajo tiene coste **constante**, que podemos denotar por una constante $ c_0 $. En notación asintótica:

$$
T(1) = \Theta(1)
$$

### 3.2. Caso recursivo (n > 1)

Si `left < right`, el tamaño del segmento es $ n > 1 $. El algoritmo realiza:

1. Cálculo de `mid = (left + right) // 2` → coste constante.

2. Una llamada recursiva sobre la mitad izquierda: tamaño aproximadamente  n/2 .

3. Una llamada recursiva sobre la mitad derecha: tamaño aproximadamente  n/2 .

4. Una suma de los resultados (`+`), también de coste constante.


Podemos descomponer el coste como:

- Coste de la primera llamada recursiva:  T(n/2) 
- Coste de la segunda llamada recursiva:  T(n/2) 
- Coste del trabajo adicional (cálculo de `mid`, suma, comparaciones de control): una constante  c .

Por tanto, para  n > 1 :

$$
T(n) = T(n/2) + T(n/2) + c = 2T(n/2) + c.
$$

En notación asintótica, podemos escribir:

$$
T(n) = 2T(n/2) + \Theta(1).
$$

Esta es la **ecuación de recurrencia** que buscábamos: describe el tiempo de ejecución del algoritmo en función del tamaño del problema  n .


## 4. Verificación empírica con un contador de llamadas

Para reforzar la interpretación de la recurrencia, podemos comprobar cómo crece el número de llamadas recursivas cuando aumenta el tamaño del vector.

Si en cada llamada se generan **dos llamadas recursivas** (salvo en el caso base) y la profundidad de la recursión es del orden de  \log_2 n , el número total de llamadas será proporcional a  n .

Añadimos un contador de llamadas al algoritmo:


In [3]:
class CallCounter:
    def __init__(self):
        self.calls = 0


def merge_count_with_counter(a, left, right, counter):
    """Versión de merge_count que cuenta cuántas llamadas recursivas se realizan."""
    counter.calls += 1  # registramos una llamada más
    
    if left == right:
        return 1 if a[left] > 0 else 0
    
    mid = (left + right) // 2
    return (merge_count_with_counter(a, left, mid, counter) +
            merge_count_with_counter(a, mid + 1, right, counter))


def count_positives_with_counter(n):
    """Genera un vector de tamaño n y cuenta llamadas de merge_count_with_counter."""
    # El contenido del vector no afecta al número de llamadas, solo al tamaño n.
    a = [1] * n
    counter = CallCounter()
    result = merge_count_with_counter(a, 0, n - 1, counter)
    return result, counter.calls


Probamos ahora el número de llamadas para distintos tamaños del vector:


In [ ]:
for n in [1, 2, 4, 8, 16, 32]:
    result, calls = count_positives_with_counter(n)
    print(f"n = {n:2d}, llamadas recursivas = {calls}, resultado = {result}")

n =  1, llamadas recursivas = 1, resultado = 1
n =  2, llamadas recursivas = 3, resultado = 2
n =  4, llamadas recursivas = 7, resultado = 4
n =  8, llamadas recursivas = 15, resultado = 8
n = 16, llamadas recursivas = 31, resultado = 16
n = 32, llamadas recursivas = 63, resultado = 32


Observando la salida, vemos que el número de llamadas crece de forma proporcional al tamaño  n .
Esto es coherente con la estructura del árbol de recursión:

- La recursión se detiene cuando los segmentos son de tamaño 1.

- El árbol de llamadas tiene altura proporcional a  $\log_2 n$

- El número total de nodos (llamadas) del árbol es del orden de  n .


Todo esto refuerza la validez de la recurrencia:

$$
T(n) = 2T(n) + c,
$$

que es característica de algoritmos de tipo **divide y vencerás** con dos subproblemas de tamaño  n/2  y trabajo adicional constante.

En actividades posteriores se utilizarán técnicas como el **Teorema Maestro** para resolver este tipo de recurrencias y obtener explícitamente que:

$$
T(n) = \Theta(n),
$$

es decir, que el algoritmo `MergeCount` tiene **complejidad lineal** respecto al tamaño del vector.
